## PART 2

In [1]:
using CSV, DataFrames
filepath = "interpolated_height.csv"
df = CSV.read(filepath, DataFrame)	
H = round.(df[:, 2], digits=0)

SyntaxError: invalid syntax (2266721729.py, line 1)

# Problem 3

In [4]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )

    @objective(myModel, Min, sum(x[j] for j=1:h) )

    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

SyntaxError: invalid syntax (3171596515.py, line 1)

# Problem 4

In [ ]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )
    @variable(myModel, z[1:h] >= 0 )

    @objective(myModel, Min, sum(z[j] for j=1:h) )

    #introduce z for the absolute value of R-H-CHD
    @constraint(myModel, [j=1:h],z[j]>=R[j]-H[j]-10 )
    @constraint(myModel, [j=1:h],z[j]>=-R[j]+H[j]+10 )
    
    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

# Problem 5

In [ ]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )
    @variable(myModel, z[1:h] >= 0 )

    @objective(myModel, Min, sum(z[j] for j=1:h) )

    #introduce z for the absolute value of R-H-CHD
    @constraint(myModel, [j=1:h],z[j]>=R[j]-H[j]-10 )
    @constraint(myModel, [j=1:h],z[j]>=-R[j]+H[j]+10 )
    
    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    # constraints for x
    @constraint(myModel, x[1] + x[2] <= 1)
    @constraint(myModel, [j=2:h-1], x[j-1] + x[j] <= 1)
    @constraint(myModel, x[h-1] + x[h] <= 1)
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)

# Problem 6

In [ ]:
using GLPK, Cbc, JuMP, SparseArrays

K = [
300 140 40
]


function constructA(H, K)
    h = length(H)
    k = length(K)
    I = Int[]
    J = Int[]
    V = Float64[]

    for i = 1:h
        for j = 1:h
            diff = i - j
            if abs(diff) < k
                push!(I, i)
                push!(J, j)
                push!(V, K[1+abs(diff)])
            end
        end
    end
    return sparse(I, J, V, h, h)
end

function solveIP(H, K)
    h = length(H)
    myModel = Model(Cbc.Optimizer)
    # If your want ot use GLPK instead use:
    #myModel = Model(GLPK.Optimizer)
    #set_optimizer_attribute(myModel, "ratioGap", 0.1)
    #set_optimizer_attribute(myModel, "seconds", 20)

    A = constructA(H,K)

    @variable(myModel, x[1:h], Bin )
    @variable(myModel, R[1:h] >= 0 )
    @variable(myModel, z[1:h] >= 0 )

    @objective(myModel, Min, sum(z[j] for j=1:h) )

    #introduce z for the absolute value of R-H-CHD
    @constraint(myModel, [j=1:h],z[j]>=R[j]-H[j]-10 )
    @constraint(myModel, [j=1:h],z[j]>=-R[j]+H[j]+10 )
    
    @constraint(myModel, [j=1:h],R[j] >= H[j] + 10 )
    @constraint(myModel, [i=1:h],R[i] == sum(A[i,j]*x[j] for j=1:h) )
    
    # constraints for x
    @constraint(myModel, x[1] + x[2] <= 1)
    @constraint(myModel, [j=2:h-1], x[j-1] + x[j] <= 1)
    @constraint(myModel, x[h-1] + x[h] <= 1)
    
    optimize!(myModel)

    if termination_status(myModel) == MOI.TIME_LIMIT && has_values(myModel)
        println("Time limit reached. Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    elseif termination_status(myModel) == MOI.OPTIMAL && has_values(myModel)
        println("Optimal solution found")
        println("Objective value: ", JuMP.objective_value(myModel))
        println("x = ", JuMP.value.(x))
        println("R = ", JuMP.value.(R))
    else
        println("Optimize was not succesful. Return code: ", termination_status(myModel))
    end
end

solveIP(H,K)